In [ ]:
# try to install ADCP processing tool

In [1]:
# Install some packages
!pip install gsw
!pip install ipywidgets

import os

### Install pycurrents
#Download 
!wget https://currents.soest.hawaii.edu/hg/pycurrents/archive/tip.zip
!unzip tip.zip 

# # Install
pycurrent_dir = [ directory for directory in os.listdir() if directory.startswith('pycurrents')][0]
pycurrent_dir

os.chdir(pycurrent_dir)
!pip install -e .
# os.chdir('/content/')

# Install pycurrents_ADCP_processing
!pip install git+https://github.com/JessyBarrette/pycurrents_ADCP_processing.git@hakai_dev

# Install process_ocean_timeseries
!pip install git+https://github.com/HakaiInstitute/process_ocean_timeseries.git

     |████████████████████████████████| 2.4 MB 7.1 MB/s 
--2021-09-29 17:03:18--  https://currents.soest.hawaii.edu/hg/pycurrents/archive/tip.zip
Resolving currents.soest.hawaii.edu (currents.soest.hawaii.edu)... 128.171.154.235
Connecting to currents.soest.hawaii.edu (currents.soest.hawaii.edu)|128.171.154.235|:443... connected.
HTTP request sent, awaiting response... 200 Script output follows
Length: unspecified [application/zip]
Saving to: ‘tip.zip’

tip.zip                 [     <=>            ]   2.19M  2.33MB/s    in 0.9s    

2021-09-29 17:03:19 (2.33 MB/s) - ‘tip.zip’ saved [2295447]

Archive:  tip.zip
  inflating: pycurrents-c22d6c40e594/.hg_archival.txt  
  inflating: pycurrents-c22d6c40e594/.hgignore  
  inflating: pycurrents-c22d6c40e594/.hgtags  
  inflating: pycurrents-c22d6c40e594/README.txt  
  inflating: pycurrents-c22d6c40e594/doc/INSTALL.txt  
  inflating: pycurrents-c22d6c40e594/doc/Makefile  
  inflating: pycurrents-c22d6c40e594/doc/conf.py  
  inflating: pycurrent

In [2]:
from process_ocean_data.tools import google, hakai, process, geo

from pycurrents_ADCP_processing import ADCP_processing_L0, ADCP_processing_L1
import xarray as xr
import pandas as pd
import numpy as np
import glob

from ipywidgets import widgets


In [3]:
# Return to Google Colab initial directory
!cd /content/

# Retrieve Hakai ADCP Deployment Log
Apply transformations to Hakai log
- Convert times to datetime objects in UTC
- Retrieve instrument time offset from UTC
- Convert lat/long to decimal degrees
- Compute trilateration if available
- Retrieve Magnetic Declination
- Generate standard Hakai File Name

In [4]:
# File path
dest_dir = r"/content/processed_data/"
# Generate output directory if doesn't exist
if not os.path.exists(dest_dir):
     os.mkdir(dest_dir)

# Define Spreadsheet ID
# Hakai ADCP Deployment log
# The ID and range of a sample spreadsheet.
INSTRUMENT_LOG_LINK = 'https://docs.google.com/spreadsheets/d/1fg8QEdZIE1sSYf4lAM-p12LaiBs4Cli8CVJriwaeAFs/edit?usp=sharing'
SAMPLE_RANGE_NAME = 'ADCP Deployments!A:AZ'

# Get Hakai Instrument Log and convert it to a dataframe
df = google.get_from_google_public(INSTRUMENT_LOG_LINK)

# Apply transformations to Hakai log
#  - Convert times to datetime objects in UTC
#  - Retrieve instrument time offset from UTC
#  - Convert lat/long to decimal degrees
#  - Compute trilateration if available
#  - Retrieve Magnetic Declination
#  - Generate standard Hakai File Name
df = hakai.transform_hakai_log(df, dest_dir,get_mag_dec=True)

# Ignore entries with no data associated
df = df.dropna(subset=['Link to Raw Data'])


Retrieve Time Zone Internal Clock Sync or Start Time
Get Magnetic Declination Values from NRCAN
Triangulate deployment location


# Generate IOS Metadata File for each deployment

In [5]:
# Produce a metadata file for each Hakai ADCP Deployment based on the metadata variable mapping show below:
hakai_to_ios_map = {'acknowledgement': 'Hakai Mooring Group',
                    'agency': 'Hakai Institute',
                    'anchor_type': 'Anchor Type',  # Need to add a column describing the deployment type,
                    'anchor_drop_time': 'Deployment Time',
                    'anchor_release_time': 'Retrieval Time',
                    'comment': 'Comments',
                    'country': 'Canada',
                    'country_institute_code': '-99',
                    'cruise_description': '',
                    'cut_lead_ensembles': 0,
                    'cut_trail_ensembles': 0,
                    'deployment_cruise_number': '',
                    'geographic_area': 'Region',
                    'history': 'Magnetic Declination was retrieve for the specific location and deployment time from '
                               'NRCAN website.',
                    'instrument_depth': '-99',
                    'latitude': 'Latitude',
                    'longitude': 'Longitude',
                    'platform': 'Platform',  # not sure what it could relate to within Hakai
                    'project': 'Project',
                    'publisher_email': 'info@hakai.org',
                    'return_cruise_number': '',
                    'scientist': 'Scientist',
                    'deployment_number': '',
                    'water_depth': '-99',
                    'instrumentSubtype': 'Instrument Sub Type',
                    'serialNumber': 'Serial Number',
                    'magnetic_variation': 'Magnetic Declination',
                    'instrument_clock_seconds_utc_offset': 'Instrument_clock_seconds_utc_offset'}

hakai.hakai_log_to_ios_csv(df, hakai_to_ios_map, dest_dir)


Empty Fields:
dict_keys(['cruise_description', 'deployment_cruise_number', 'return_cruise_number', 'deployment_number'])
From Hakai Log:
dict_keys(['anchor_type', 'anchor_drop_time', 'anchor_release_time', 'comment', 'geographic_area', 'latitude', 'longitude', 'platform', 'project', 'scientist', 'instrumentSubtype', 'serialNumber', 'magnetic_variation', 'instrument_clock_seconds_utc_offset'])
Default Values:
{'acknowledgement': 'Hakai Mooring Group', 'agency': 'Hakai Institute', 'country_institute_code': '-99', 'water_depth': '-99', 'instrument_depth': '-99', 'publisher_email': 'info@hakai.org', 'history': 'Magnetic Declination was retrieve for the specific location and deployment time from NRCAN website.', 'country': 'Canada', 'cut_trail_ensembles': 0, 'cut_lead_ensembles': 0}
Name
acknowledgement                                                      Hakai Mooring Group
agency                                                                   Hakai Institute
anchor_type                 

In [6]:
# Retrieve Raw ADCP data from google drive (*.000)
raw_file_list = []
for id,row in df.iterrows():
    raw_file = os.path.join(dest_dir,row['file_name']+'.000')
    google.get_from_google_public(row['Link to Raw Data'], 
                                  os.path.join(dest_dir,row['file_name']+'.000'))
    raw_file_list.append(raw_file)

Download /content/processed_data/Hakai_RDI-ADCP-Workhorse-SN24621_Quadra-QuadraADCP_20190222-20190408.000: : 58.53125MB [00:00, 169.18MB/s]
Download /content/processed_data/Hakai_RDI-ADCP-Workhorse-SN24621_Calvert-KC6M_20190514-20190719.000: : 60.46875MB [00:00, 129.95MB/s]
Download /content/processed_data/Hakai_RDI-ADCP-Workhorse-SN24621_Calvert-KC6M_20190721-20190930.000: : 67.28125MB [00:00, 143.16MB/s]
Download /content/processed_data/Hakai_RDI-ADCP-Workhorse-SN24621_Calvert-KC6M_20191005.000: : 117.65625MB [00:01, 103.66MB/s]
Download /content/processed_data/Hakai_RDI-ADCP-Workhorse-SN24621_Calvert-KC6M_20200212-20200828.000: : 162.6875MB [00:01, 110.02MB/s] 
Download /content/processed_data/Hakai_RDI-ADCP-Workhorse-SN18834_Calvert-Surf Pass_20200703-20200703.000: : 1.84375MB [00:00, 101.79MB/s]
Download /content/processed_data/Hakai_RDI-ADCP-Workhorse-SN18834_Calvert-West Beach_20200704.000: : 1.46875MB [00:00, 107.04MB/s]
Download /content/processed_data/Hakai_RDI-ADCP-Workhorse

In [ ]:
# Process each files automatically
for raw_file in raw_file_list:
    meta_file = raw_file[0:-4] + '_meta.csv'

    # Perform Initial L0 processing on the raw data and export as a netCDF file
    ncname_L0 = ADCP_processing_L0.nc_create_L0(f_adcp=raw_file, f_meta=meta_file, dest_dir=dest_dir)

    # Read Level 0 Data
    ds = xr.open_dataset(ncname_L0)

    # Detect start and end times
    #  CMAGZZ## should have some good data
    cmag_min_acceptable_value = 64
    is_good_data = ((ds['CMAGZZ01'] > cmag_min_acceptable_value).any('distance') &
                    (ds['CMAGZZ02'] > cmag_min_acceptable_value).any('distance') &
                    (ds['CMAGZZ03'] > cmag_min_acceptable_value).any('distance') &
                    (ds['CMAGZZ04'] > cmag_min_acceptable_value).any('distance'))
    start_end_results = process.detect_start_end(ds, 'time', 'PRESPR01', is_good_data,
                                                 figure_path=ncname_L0[0:-3]+'_start_end.png')
    # Close data set
    ds.close()

    # Update metadata to reflect results
    # Read L0 meta and update fields for Level 1 processing
    df_meta_l1 = pd.read_csv(raw_file[0:-4]+'_meta.csv').set_index('Name')

    # Update fields
    df_meta_l1.loc['cut_lead_ensembles', 'Value'] = start_end_results['cut_lead_ensembles']
    df_meta_l1.loc['cut_trail_ensembles', 'Value'] = start_end_results['cut_trail_ensembles']-1
    df_meta_l1.loc['instrument_depth'] = start_end_results['instrument_depth']

    # Add new fields
    df_meta_l1['pressure_offset_at_deployment'] = start_end_results['pressure_offset_deployment']
    df_meta_l1['pressure_offset_at_retrieval'] = start_end_results['pressure_offset_retrieval']

    # Save metadata to L1 metadata
    meta_l1 = raw_file[0:-4]+'_meta_L1.csv'
    df_meta_l1.to_csv(meta_l1)

    # Process L1 Data
    ncname_L1 = ADCP_processing_L1.nc_create_L1(inFile=raw_file, file_meta=meta_l1, dest_dir=dest_dir)

    # Read Level 1 dataset and extra steps
    ds = xr.open_dataset(ncname_L1)

    # If deployment is less than a day apply pressure offset
    if ds['time'][[0, -1]].diff('time').dt.days.values[0] < 1:
        pressure_offset = np.mean([df_meta_l1['pressure_offset_at_deployment'],df_meta_l1['pressure_offset_at_retrieval']])
        ds['PPSAADCP'] = ds['PPSAADCP'] - pressure_offset
        ds.attrs['history'] +=f'- Substract {pressure_offset:0.3f}dbar from Pressure Measurement'

    # Derive Depth from surface variable
    ds['depth'] = -(ds['distance'] - ds['PPSAADCP'])
    ds['depth'].attrs['units'] = 'm'
    ds['depth'].attrs['standard_name'] = 'depth'

    # Flag Currents values
    current_flag_variables = ['LCEWAP01_QC', 'LCNSAP01_QC', 'LRZAAP01_QC']
    
    # Flag out of water data
    for var in current_flag_variables:
        ds[var] = ds[var].where(ds['depth'] > 0, other=4)

    # Flag side lobe region from surface data
    side_lobe_coefficient = 1 - np.cos(np.deg2rad(int(ds.attrs['beam_angle'])))
    side_lobe_depth = ds['PPSAADCP'] * side_lobe_coefficient + ds.attrs['cellSize']/2
    for var in current_flag_variables:
        # Flag BAD any data above side lobe contaminated region
        ds[var] = ds[var].where((ds['depth'] >= side_lobe_depth), other=4)

        # Flag BAD any data above surface
        ds[var] = ds[var].where(ds['depth'] > 0, other=4)

        # Flag GOOD any data unflagged yet
        ds[var] = ds[var].where(ds[var].notnull(),other=1)

    # Replace None which is not compatible with xarray
    ds.attrs['_FillValue'] = np.nan

    # Save to a new NetCDF File
    ncname_L1_hakai = ncname_L1[0:-3] + '_Hakai.nc'
    ds.to_netcdf(ncname_L1[0:-3] + '_Hakai.nc', mode='w', format='NETCDF4')
    ds.close()


Hakai_RDI-ADCP-Workhorse-SN24621_Quadra-QuadraADCP_20190222-20190408.adcp.L0.nc
Read in csv metadata file
Read in raw data
0
0
0
Assigning pressure variable
Assigning percent good variables
Pressure Offset [pre, post] = [0.017000000923871994, -0.11100000143051147]
Hakai_RDI-ADCP-Workhorse-SN24621_Quadra-QuadraADCP_20190222-20190408.adcp.L1.nc
wh
Read in csv metadata file
Read in raw data
np.max(counts): 41
counts[index_of_zero]: [2]
serial number: WH24621
Calculated sea surface height from sea pressure using gsw package
(53171, 50) (53171, 50)
[[0.14506575 0.15889837        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 ...
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]]
[[4. 4

# Review File


In [144]:
from ipywidgets import widgets, interact
import plotly.express as px
import matplotlib.pyplot as plt

In [145]:
files = os.listdir(dest_dir)
files = [file for file in files if file.endswith('.nc')]

file_selector = widgets.Dropdown(
    options=sorted(files),
    value=files[0],
    description='Select File to Review:',
    disabled=False,
)
file_selector

Dropdown(description='Select File to Review:', index=31, options=('Hakai_RDI-ADCP-Workhorse-SN18834_Calvert-St…

In [147]:
def plot_adcp(file,variable, apply_flag):
    ds = xr.open_dataset(os.path.join(dest_dir,file_selector.value))
    fig,ax = plt.subplots(ncols=1)
    fig.set_size_inches(15,6)
    try:    
        if variable+'_QC' in ds and apply_flag:
            if len(ds[variable].dims)>1:
                ds[variable].where(ds[variable+'_QC']==1,drop=True).plot(robust=True,axes=ax)
            else:
                ds[variable].where((ds['LCEWAP01_QC']==1).any(dim='distance'),drop=True).plot(axes=ax)
        else:
            ds[variable].plot(robust=True,axes=ax)
    except:
        ds[variable].plot(axes=ax)
    
    plt.tight_layout()
    plt.draw()

interact(plot_adcp,file=file_selector,variable=ds.keys(),apply_flag=True)

interactive(children=(Dropdown(description='Select File to Review:', index=14, options=('Hakai_RDI-ADCP-Workho…

<function __main__.plot_adcp>